In [ ]:
# Partagé par Nicolas Ollinger

from p5 import *
from IPython.display import HTML

colors = {
    "W": (255, 255, 255),
    "R": (137, 18, 20),
    "B": (13, 72, 172),
    "O": (255, 85, 37),
    "G": (25, 155, 76),
    "Y": (254, 213, 47),
    "X": (76, 76, 76),
}

sroloc = {}


def micolor(rgb):
    return tuple([(2 * x + 200) // 3 for x in rgb])


for k in colors:
    sroloc[k] = micolor(colors[k])


def toggleColors():
    global colors, sroloc
    colors, sroloc = sroloc, colors


faceorder = "udlrfb"

transface = {
    "f": lambda: None,
    "r": lambda: rotateY(270),
    "b": lambda: rotateY(180),
    "l": lambda: rotateY(90),
    "u": lambda: rotateX(90),
    "d": lambda: rotateX(270),
}


def coord(x, y, z):
    return 16 * x + 4 * y + z


facemask = {
    "f": (coord(0, 0, 3), coord(0, 0, 0)),
    "2f": (coord(0, 0, 3), [coord(0, 0, 0), coord(0, 0, 1)]),
    "s": (coord(0, 0, 3), coord(0, 0, 1)),
    "b": (coord(0, 0, 3), coord(0, 0, 2)),
    "2b": (coord(0, 0, 3), [coord(0, 0, 2), coord(0, 0, 1)]),
    "d": (coord(0, 3, 0), coord(0, 0, 0)),
    "2d": (coord(0, 3, 0), [coord(0, 0, 0), coord(0, 1, 0)]),
    "e": (coord(0, 3, 0), coord(0, 1, 0)),
    "u": (coord(0, 3, 0), coord(0, 2, 0)),
    "2u": (coord(0, 3, 0), [coord(0, 2, 0), coord(0, 1, 0)]),
    "l": (coord(3, 0, 0), coord(0, 0, 0)),
    "2l": (coord(3, 0, 0), [coord(0, 0, 0), coord(1, 0, 0)]),
    "m": (coord(3, 0, 0), coord(1, 0, 0)),
    "r": (coord(3, 0, 0), coord(2, 0, 0)),
    "2r": (coord(3, 0, 0), [coord(2, 0, 0), coord(1, 0, 0)]),
    "x": (0, 0),
    "y": (0, 0),
    "z": (0, 0),
    " ": (0, 0),
}

faceidx = {
    "f": (coord(0, 0, 0), coord(1, 0, 0), coord(0, 1, 0)),
    "s": (coord(0, 0, 1), coord(1, 0, 0), coord(0, 1, 0)),
    "b": (coord(2, 0, 2), coord(-1, 0, 0), coord(0, 1, 0)),
    "u": (coord(0, 2, 0), coord(1, 0, 0), coord(0, 0, 1)),
    "e": (coord(0, 1, 0), coord(1, 0, 0), coord(0, 0, 1)),
    "d": (coord(0, 0, 2), coord(1, 0, 0), coord(0, 0, -1)),
    "l": (coord(0, 0, 2), coord(0, 0, -1), coord(0, 1, 0)),
    "m": (coord(1, 0, 0), coord(0, 0, 1), coord(0, 1, 0)),
    "r": (coord(2, 0, 0), coord(0, 0, 1), coord(0, 1, 0)),
}


def facecoord(f, x, y):
    ze, dx, dy = faceidx[f]
    return ze + dx * x + dy * y


def turnaround(face):
    for k in range(3):
        yield 0, facecoord(face, k, 0)
    for k in range(3):
        yield 1, facecoord(face, 2, k)
    for k in range(3):
        yield 2, facecoord(face, 2 - k, 2)
    for k in range(3):
        yield 3, facecoord(face, 0, 2 - k)


facesides = {
    "f": "drul",
    "s": "drul",
    "b": "dlur",
    "u": "frbl",
    "e": "frbl",
    "d": "brfl",
    "l": "dfub",
    "m": "dbuf",
    "r": "dbuf",
}

faceanim = {
    "f": lambda x: rotateZ(-x),
    "z": lambda x: rotateZ(-x),
    "s": lambda x: rotateZ(-x),
    "b": lambda x: rotateZ(x),
    "l": lambda x: rotateX(-x),
    "m": lambda x: rotateX(x),
    "r": lambda x: rotateX(x),
    "x": lambda x: rotateX(x),
    "u": lambda x: rotateY(x),
    "y": lambda x: rotateY(x),
    "e": lambda x: rotateY(x),
    "d": lambda x: rotateY(-x),
    " ": lambda x: None,
}

combos = {
    "x": "rmlll",
    "y": "ueddd",
    "z": "fsbbb",
    "2u": "ue",
    "2d": "deee",
    "2r": "rm",
    "2l": "lmmm",
    "2f": "fs",
    "2b": "bsss",
}

HTML("<em>Définitions longues et ennuyeuses</em>")

In [ ]:
class Sticker:
    def __init__(self, color, idx):
        self.color = color
        self.idx = idx

    def __repr__(self):
        return f"Sticker({self.color},{self.idx})"

    def draw(self):
        noStroke()
        fill(*colors[self.color])
        plane(0.9, 0.9)


class Cubie:
    def __init__(self, x, y, z, vis=True):
        self.pos = (x, y, z)
        self.stickers = {}
        self.vis = vis

    def draw(self):
        if self.vis:
            push()
            translate(*self.pos)
            fill(0, 0, 0)
            stroke(0)
            strokeWeight(0.5)
            box(1)
            for face in self.stickers:
                self.drawSticker(face)
            pop()

    def setSticker(self, face, color, idx=None):
        self.stickers[face] = Sticker(color, idx)

    def drawSticker(self, face):
        push()
        transface[face]()
        translate(0, 0, -0.501)
        self.stickers[face].draw()
        pop()

In [ ]:
def parsemove(s):
    pos = 0
    while pos < len(s):
        c = s[pos]
        if c == "2":
            pos += 1
            c += s[pos]
        yield c
        pos += 1


class Move:
    def __init__(self, move, tempo=190):
        self.move = move
        self.tempo = tempo
        self.face = move.lower()
        self.CW = self.face == move
        self.mask, self.mval = facemask[self.face]
        if type(self.mval) is not list:
            self.mval = [self.mval]
        self.angle = 0
        self.lastFrame = 0

    def update(self):
        if frameCount > self.lastFrame:
            self.lastFrame = frameCount
            self.angle += 3.0 * self.tempo * deltaTime / 2000.0
            if self.angle > 90:
                return True
        return False

    def animate(self, idx):
        if idx & self.mask in self.mval:
            faceanim[self.face[-1]](self.angle * (1 if self.CW else -1))

In [ ]:
class Cube:
    def __init__(
        self,
        moves="",
        stickers={"u": "W", "d": "Y", "l": "B", "r": "G", "f": "O", "b": "R"},
        tempo=190,
    ):
        self.tempo = tempo
        self.cubies = [None] * 64
        for x in range(3):
            for y in range(3):
                for z in range(3):
                    self.cubies[coord(x, y, z)] = Cubie(x - 1, y - 1, z - 1)
        for idx, face in enumerate(faceorder):
            stick = stickers[face]
            if len(stick) == 1:
                stick = stick * 9
            for x in range(3):
                for y in range(3):
                    self.cubies[facecoord(face, x, y)].setSticker(
                        face, stick[3 * y + x], 9 * idx + 3 * y + x
                    )
        self.setMoves(moves)

    def colors(self):
        stickers = {}
        for face in faceorder:
            stickers[face] = "".join(
                [
                    self.cubies[facecoord(face, x, y)].stickers[face].color
                    for y in range(3)
                    for x in range(3)
                ]
            )
        return stickers

    def setMoves(self, moves):
        self.moves = parsemove(moves)
        self.curMove = None
        self.moving = True
        self.updateMove()

    def updateMove(self):
        if self.curMove is None:
            move = next(self.moves, None)
            if move is not None:
                self.curMove = Move(move, self.tempo)
        self.moving = self.curMove is None

    def rotate(self, moves):
        for move in parsemove(moves):
            lmove = move.lower()
            CW = move == lmove
            seq = combos.get(lmove, lmove)
            for r in range(1 if CW else 3):
                for x in seq:
                    if x in "udlrfbmes":
                        self.rotCW(x)

    def rotCW(self, face):
        todo = [facesides[face]]
        if face in "udlrfb":
            todo.append([face] * 4)
        for colset in todo:
            cols = [
                self.cubies[c].stickers[colset[side]] for side, c in turnaround(face)
            ]
            for (side, c), col in zip(turnaround(face), cols[3:] + cols[:3]):
                self.cubies[c].stickers[colset[side]] = col

    def draw(self):
        push()
        self.updateMove()
        if self.curMove is not None:
            if self.curMove.update():
                self.rotate(self.curMove.move)
                self.curMove = None
        for idx, c in enumerate(self.cubies):
            if c is not None:
                push()
                if self.curMove is not None:
                    self.curMove.animate(idx)
                c.draw()
                pop()
        pop()

In [ ]:
class Player:
    def __init__(self, cube=Cube(), mirror=True):
        self.mirror = mirror
        self.cube = cube

    def run(self):
        run(self.setup, self.draw)

    def setup(self):
        createCanvas(800, 600, WEBGL)
        angleMode(DEGREES)

    def drawMirror(self, axe, delta):
        push()
        translate(*[delta if a == axe else 0 for a in range(3)])
        scale(*[-1 if a == axe else 1 for a in range(3)])
        self.cube.draw()
        pop()

    def draw(self):
        background(200)
        scale(50, -50, -50)
        rotateX(-30)
        rotateY(40)
        translate(0.5, 2.5, 0)

        self.cube.draw()
        if self.mirror:
            toggleColors()
            self.drawMirror(0, -8)
            self.drawMirror(0, 8)
            self.drawMirror(1, -6)
            self.drawMirror(1, 6)
            self.drawMirror(2, 12)
            self.drawMirror(2, -12)
            toggleColors()
        orbitControl()

In [ ]:
cube = Cube("fruRUF 2r xyz")
player = Player(cube)
# cube = Cube('rrurrUrrUdrrUrrurr',{'u': 'Y', 'd': 'X', 'l': 'XXXXXXBBB', 'r': 'XXXXXXGGG', 'f': 'XXXXXXOOO', 'b': 'XXXXXXRRR'})

In [ ]:
player.run()

In [ ]:
player.cube.setMoves("ldLD")